# Библиотеки для анализа данных. NumPy и SciPy

In [ ]:
# НИКОГДА НЕ ДЕЛАЙТЕ ТАК. ВООБЩЕ НИКОГДА
# from numpy import *

import numpy as np

In [ ]:
from itertools import chain, zip_longest


def print_as_columns(*args, sep='\t'):
    """
    print arrays as columns
    """
    args = map(repr, args)
    args = list(map(lambda s: s.split('\n'), args))
    width = max(map(len, chain.from_iterable(args)))
    
    fill = lambda s: '{:<{width}s}'.format(s, width=width)
    fillvalue = fill('')
    
    args = map(lambda e: map(fill, e), args)
    args = map(sep.join, zip_longest(*args, fillvalue=fillvalue))
    print(*args, sep='\n')

## Многомерные массивы в Python

Здесь и далее будем считать, что понятия "многомерные массивы" и "матрицы" эквивалентны.

Многомерные массивы в Python можно рассмаривать, например, как списки списков:

In [ ]:
a = [[ 0,  1,  2,  3,  4],
     [ 5,  6,  7,  8,  9],
     [10, 11, 12, 13, 14],
     [15, 16, 17, 18, 19]]
a

In [ ]:
a[0][2]

In [ ]:
rows, cols = 1_000, 1_000

A = [list(range(i * cols, (i + 1) * cols)) for i in range(0, rows)]

# show first 5 rows and first 10 cols
[row[:10] for row in A[:5]]

In [ ]:
%%timeit B = [[0 for j in range(cols)] for i in range(rows)]

# list cycles

for i in range(rows):
    for j in range(cols):
        B[i][j] = 2 * A[i][j]

In [ ]:
%%timeit B = [[0 for j in range(cols)] for i in range(rows)]

# list cycles with optimization

for i in range(rows):
    A_row, B_row = A[i], B[i]
    for j in range(cols):
        B_row[j] = 2 * A_row[j]

In [ ]:
%%timeit

# list generator

B = [[2 * e for e in row] for row in A]

In [ ]:
%%timeit A_numpy = np.asarray(A)

B = 2 * A_numpy

Таким образом, главным преимуществом библиотеки `numpy` являтся высокая скорость работы: ускорение почти в 50 раз по сравнению с лучшим вариантом на python.

| Реализация              | Время работы |
|-------------------------|--------------|
| lists cycles            | 131 ms       |
| lists cycles (optimize) | 88.7 ms      |
| lists generators        | 68.9 ms      |
| numpy                   | 1.54 ms      |

Самое медленное место в языке – циклы. **Думайте** о количестве операций (потенциальных вызовах функций).

## Создание массивов

Массив `numpy.ndarray` имеет фиксированные размеры и фиксированный тип данных.

Вариант 1: создание `numpy.ndarray` заданных форм и характеристик:
* `np.zeros` – матрица из нулей;
* `np.ones` – матрица из единицы;
* `np.eye` – единичная марица;
* `np.arange`, `np.linspace` – массивы с дискретным шагом.

https://numpy.org/doc/stable/reference/routines.array-creation.html

In [ ]:
np.zeros(5)

In [ ]:
np.zeros((5, ))

In [ ]:
np.zeros((5, 1))

In [ ]:
# похоже на range, но все числа хранятся в памяти

np.arange(10)

In [ ]:
# а еще умеет ходить с дробным шагом

np.arange(2, 8, 0.5)

In [ ]:
# разбивает отрезок на num - 1 отрезков

np.linspace(0, 1, 5)

Вариант 2: создание из python объектов

In [ ]:
a = [[ 0,  1,  2,  3,  4],
     [ 5,  6,  7,  8,  9],
     [10, 11, 12, 13, 14],
     [15, 16, 17, 18, 19]]

a = np.asarray(a, dtype=int)
a

Есть и обратная операция.

In [ ]:
a = a.tolist()
print(type(a))
a

### Важное замечание о скорости работы

In [ ]:
%%timeit A_numpy = np.asarray(A)

B = 2 * A_numpy

In [ ]:
%%timeit

A_numpy = np.asarray(A)
B = 2 * A_numpy

In [ ]:
%%timeit

# list generator

B = [[2 * e for e in row] for row in A]

## Поэлементные операции со скалярами

In [ ]:
a = [[ 0,  1,  2,  3,  4],
     [ 5,  6,  7,  8,  9],
     [10, 11, 12, 13, 14],
     [15, 16, 17, 18, 19]]

a = np.asarray(a, dtype=int)
a

`numpy.ndarray` поддерживает арифметические поэлементые операции:

`c = op(a, sc)` $\Longleftrightarrow$ `c[i,j] = op(a[i,j], sc)`

In [ ]:
a + 10   # np.add

In [ ]:
a * 10

In [ ]:
a / 10

In [ ]:
a // 10

Есть и поддержка более сложных математических функций.

In [ ]:
a ** 0.5  # same as np.pow(a, 2)

In [ ]:
np.sqrt(a)

In [ ]:
np.sin(a)

И поддержка логических операций.

In [ ]:
a >= 10

## Устройство массивов, их размерность и размеры

<img src="files/data.png" width="450px">

In [ ]:
a = np.array([[1, 2, 3, 4, 5],
              [6, 7, 8, 9, 0]], 
             dtype=float)
a

Массивы `numpy.ndarray` в основе себя используют обычные массивы фиксированного размера (атрибут `data`). 

Многомерные массивы хранятся как один большой последовательный блок памяти.

In [ ]:
a.data

У `numpy.ndarray` есть атрибут `dtype`, который содержит информацию о типе данных, хранимых в массиве.

In [ ]:
a.dtype

In [ ]:
a.dtype.itemsize  # sizeof(float64)

Поменять тип массива можно с помощью метода `astype`, что очевидно приведет копированию данных.

In [ ]:
a.astype(int).dtype

In [ ]:
a.astype(np.uint8).dtype

У `numpy.ndarray` есть атрибут `shape` – размер массива.

In [ ]:
a.shape

Для изменения размера массива используется метод `reshape`. Метод возвращает новый объект, который имеет тот же атрибут `data`, что и исходный массив. Т.е. копирования данных **НЕ происходит**.

In [ ]:
a.reshape(5, 2)

In [ ]:
# размеры должны быть корректными

a.reshape(5, 3)

In [ ]:
# -1 говорит о том, что длина размерности может быть вычислена автоматически

a.reshape(-1, 2)

In [ ]:
a.ravel()  # same as a.reshape(-1)

In [ ]:
a.flatten()   # same as a.ravel() and copy

Еще раз подчеркнем, что копирования данных **не происходит**!

In [ ]:
a

In [ ]:
b = a.reshape(-1, 2)
b

In [ ]:
a[-1, -1] = 10
a

In [ ]:
b

Другой способ в этом убедиться – получить адрес памяти:

https://numpy.org/doc/stable/reference/generated/numpy.ndarray.ctypes.html

In [ ]:
a.ctypes.data, b.ctypes.data, a.ctypes.data == b.ctypes.data

In [ ]:
a = np.arange(2 * 3 * 4).reshape(-1, 3, 4)
a

In [ ]:
a.shape

У массивов `numpy.ndarray` есть операция транспонирования. Она тоже **НЕ приводит** к копированию данных. 

In [ ]:
a.T

In [ ]:
a.transpose()

Фиктивная размерность – размерность длины 1. Фиктивная размерность нужна для того, чтобы совершать матричные операции, например, умножение вектора на матрицу.

In [ ]:
a = a.ravel()
a

In [ ]:
a.shape

In [ ]:
b = a[np.newaxis, :]  # same as a.reshape(1, -1)
b

In [ ]:
b.shape

In [ ]:
b = a[:, np.newaxis]  # same as a.reshape(-1, 1)
b

In [ ]:
b.shape

### 🧐 Интересный факт про размерность массивов и копирование

У `numpy.ndarray` есть атрибут `strides`, который регулирует порядок обхода массива (атрибут `data`). Этот атрибут очень важен для многомерных массивов.

Первое число показывает, сколько байт нужно прочитать, чтобы переместиться на следующую строку и остаться в том же столбце.

Второе число показывает, сколько байт нужно прочитать, чтобы переместиться на следующий столбец и остаться в той же строке.

In [ ]:
c = np.arange(12).reshape(3, -1)
c

In [ ]:
c.shape

In [ ]:
c.dtype.itemsize

In [ ]:
assert c.strides == (c.shape[1] * c.dtype.itemsize, c.dtype.itemsize,)

# для перехода к следующей строке нужно считать 4 * 8 байт
# для перехода к следующему стобцу нужно прочитать 8 байт

c.strides  # (4 * 8, 8)

Поменяем размерность массива и увидим, как меняется атрибут `strides`.

In [ ]:
c = c.reshape(-1, 2)
c

In [ ]:
c.dtype

In [ ]:
c.shape

In [ ]:
assert c.strides == (c.shape[1] * c.dtype.itemsize, c.dtype.itemsize,)

# для перехода к следующей строке нужно считать 2 * 8 байт
# для перехода к следующему стобцу нужно прочитать 8 байт

c.strides  # (2 * 8, 8)

Операция транспонирования, как уже говорилось выше не приводит к копированию данных. Она меняет атрибут `strides`, который регулирет порядок обхода массива.

Напомним, что `c.data = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]`.

In [ ]:
c = c.T
c

In [ ]:
c.shape

In [ ]:
assert c.strides == (c.dtype.itemsize, c.shape[0] * c.dtype.itemsize, )

# для перехода к следующей строке нужно считать 8 байт
# для перехода к следующему столбцу нужно прочитать 2 * 8 байт

c.strides  # (8, 2 * 8)

In [ ]:
a

In [ ]:
b = c.reshape(-1, 2)
b

In [ ]:
id(c), id(b)

### Копия и представление (view)

view – новый объект, ссылающийся на те же данные, что и исходный массив.

Существует несколько вариантов создания массивов с помощью функций `np.array` и `np.asarray`.

`np.array` создает копию массива всегда.

`np.asarray` создает копию при необходимости.

1. При создании массива из списка всегда **создается копия**.

In [ ]:
print(str(b))

In [ ]:
print(repr(b))

In [ ]:
a = [1, 2, 3, 4, 5, 6]
b = np.array(a)
c = np.asarray(a)

print("b =", repr(b))
print("c =", repr(c))

In [ ]:
a[0] = 0

print("b =", repr(b))
print("c =", repr(c))

2. При создании массива из другого массива **копия не создается**.

In [ ]:
a = np.asarray([1, 2, 3, 4, 5, 6])
b = np.array(a)
c = np.asarray(a)

print("b =", repr(b))
print("c =", repr(c))

In [ ]:
a[0] = 0

print("b =", repr(b))
print("c =", repr(c))

3. При создании массива из другого массива и смены типа данных **создается копия**.

In [ ]:
a = np.asarray([1, 2, 3, 4, 5, 6], dtype=np.int8)
b = np.array(a, dtype=np.uint8)
c = np.asarray(a, dtype=np.uint8)

print("b =", repr(b))
print("c =", repr(c))

In [ ]:
a[0] = 0

print("b =", repr(b))
print("c =", repr(c))

`np.asarray` полезен тогда, когда вы хотите все время работать с `np.ndarray`, но хотите создавать копию данных только в тех случаях, когда это необходимо.

In [ ]:
def multiply_by(data, by):
    data = np.asarray(data)
    return data * by

In [ ]:
multiply_by(np.arange(4), 2)

In [ ]:
multiply_by(range(4), 2)

In [ ]:
multiply_by([0, 1, 2, 3], 2)

Некоторые операции сами по себе создают копии, а некоторые view.

https://www.jessicayung.com/numpy-views-vs-copies-avoiding-costly-mistakes/

## Поэлементные бинарные операции

In [ ]:
a = [[1, 0, 0, 0, 1], [1, 0, 1, 0, 1]]
a = np.asarray(a)
a

In [ ]:
b = np.arange(1, 11).reshape(-1, 5)
b

Для начала рассмотрим массивы одинаковых размеров.

`numpy.ndarray` поддерживает арифметические поэлементые операции над двумя массивами:

`c = op(a, b)` $\Longleftrightarrow$ `c[i,j] = op(a[i,j], b[i,j])`

In [ ]:
a + b   # same as np.add

In [ ]:
a * b   # same as np.multiply

И некоторые более сложные функции...

In [ ]:
(b <= 5).astype(int)

In [ ]:
np.fmax(a, (b <= 5).astype(int))   # element-wise maximum

Есть и поддержка булевых операций.

In [ ]:
a == (b <= 5).astype(int)

Есть поддержка матричного умножения.

In [ ]:
a @ b.T  # same as np.matmul(a, b.T)

In [ ]:
np.dot(a, b.T)  # same as a.dot(b.T)

Что еще умеют бинарные `ufunc`:

https://jakevdp.github.io/PythonDataScienceHandbook/02.03-computation-on-arrays-ufuncs.html#Advanced-Ufunc-Features

## Приведение размерностей (Broadcasting)

Теперь рассмотрим массивы с разными размерностями.

Пусть, нам дана матрица $X$ размером $5 \times 10$ и вектор $y$ длины 10. Пусть, мы хотим прибавить вектор к каждой строке матрицы. 

In [ ]:
x = np.arange(50).reshape(-1, 10)
x

In [ ]:
y = np.arange(10) * 10
y

In [ ]:
x + y

Наивный способ работает верно.

Поменяем условие, пусть, $y$ – вектор длины 5. Пусть, мы хотим прибавить вектор к каждому столбцу матрицы.

In [ ]:
y = np.arange(5) * 10
y

In [ ]:
x + y

In [ ]:
y.reshape(-1, 1)

Наивный способ не работает. Попробуем воспользоваться фиктивной размерностью.

In [ ]:
x + y[:, np.newaxis]  # same as x + y.reshape(-1, 1)

In [ ]:
a = np.arange(10)
a.shape

In [ ]:
a.reshape(-1, 1)

In [ ]:
a.reshape(-1, 1).shape

**Правило приведения размерностей (broadcasting):**

1. Предположим, что `a.shape = (a_1, a_2, ..., a_n)` и `b.shape = (b_1, b_2, ..., b_n)`. Над `a` и `b` можно произвести поэлементую бинарную операцию, если $\forall \; i \in \overline{1..n}$ выполнено хотя бы одно из условий:
    * `a_i == b_i`;
    * `a_i == 1`;
    * `b_i == 1`.


2. Если число размерностей не совпадают, то к массиву меньшей размерности добавляются ведущие фиктивные размерности. 

Документация: https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html

Проверить явное приведение размерности можно через функции `np.broadcast_to` и `np.broadcast_arrays`.

<font color='red'>**Задача.** Какие из этих команд будут выполняться с ошибкой?</font>

1. `np.ones((2, 3)) + np.ones(3)`

2. `np.ones(2) + np.ones((2, 3))`

3. `np.zeros((4, 3)) + np.ones((4, 1))`

4. `np.zeros((3, 4)) + np.ones((4, 3))`

5. `np.zeros((1, 3, 5)) + np.zeros((1, 3))`

6. `np.zeros((5, 3, 1)) + np.zeros((1, 5))`

**Как происходит бинарная операция?**

1. Одинаковые размерности.

In [ ]:
np.random.seed(36)

a = np.arange(20).reshape(4, -1)
b = 10 * np.random.randint(-1, 2, size=(4, 5))

print_as_columns(a, b)

In [ ]:
a * b

2. Одинаковое число размерностей, но среди размерностей есть фикивные.

In [ ]:
np.random.seed(36)

a = np.arange(20).reshape(4, -1)
b = 10 * np.random.randint(-1, 2, size=(4, 1))

print_as_columns(a, b)

In [ ]:
c = np.repeat(b, a.shape[1], axis=1)

print_as_columns(a, c)

In [ ]:
a * c

In [ ]:
np.allclose(a * c, a * b)

3. Разное число размерностей.

In [ ]:
np.random.seed(36)

a = np.arange(20).reshape(4, -1)
b = 10 * np.random.randint(-1, 2, size=(5, ))

print_as_columns(a, b)

a.shape, b.shape

In [ ]:
c = b[np.newaxis, :]

print_as_columns(a, c)

In [ ]:
d = np.repeat(c, a.shape[0], axis=0)

print_as_columns(a, d)

In [ ]:
a * d

In [ ]:
np.allclose(a * d, a * b)

## Операции над булевыми массивами

In [ ]:
a = np.asarray([ True,  True, False, False,  True])
b = np.asarray([ True, False, False,  True, False])
c = np.asarray([False,  False, True,  True, False])

a, b

In [ ]:
np.logical_not(a)

In [ ]:
~a   # same as np.bitwise_not(a)

In [ ]:
np.logical_and(a, b), np.logical_or(a, b), np.logical_xor(a, b)

In [ ]:
(
    a & b,  # same as np.bitwise_and(a, b)
    a | b,  # same as np.bitwise_or(a, b)
    a ^ b,  # same as np.bitwise_xor(a, b)
)

In [ ]:
a & (b | c)

## Агрегирующие операции

In [ ]:
np.random.seed(5555)

a = np.random.randint(0, 10, size=(3, 7))
a[1, 3] = 15

a

Для `numpy.ndarray` есть поддержка аггрегирующих операций: `min`, `max`, `argmin`, `argmax`, `sum`, `prod`, `mean`, `std`, `var` и др.

In [ ]:
a.min(), a.max(), a.sum(), a.prod(), a.mean()

In [ ]:
np.min(a), np.max(a), np.sum(a), np.prod(a), np.mean(a)

<img src="files/axis.png" width="350px">

`a.agg(axis=axis)` – агрегирующая операция вдоль размерности (оси) `axis`:
* выполняет редукцию (агрегирующую операцию) по размерности (оси) `axis`;
* удаляет размерности (ось) `axis` из исходного массива (аргумент `keepdims=False`).

`axis=0` – размерность строк, `axis=1` – размерность столбцов.

In [ ]:
a

In [ ]:
a.min(axis=0)

In [ ]:
a.sum(axis=1, keepdims=True)

In [ ]:
a.argmax(axis=1)

Функция `np.argAGG(a)` возвращает позицию, где достигается `AGG(a)` в вытянутом массиве `a.ravel()`. Для получения индекса в исходном массиве нужно использовать `np.unravel_index`.

In [ ]:
a.argmax()

In [ ]:
a.ravel()[a.argmax()]

In [ ]:
np.unravel_index(np.argmax(a), a.shape)

In [ ]:
a[np.unravel_index(np.argmax(a), a.shape)]

Для сравнения чисел с плавающей точкой **НУЖНО** использовать `np.isclose`.

In [ ]:
np.random.seed(4987)

a = np.random.random(size=(2, 5))
b = a + np.random.random(size=(2, 5)) * 1e-5

In [ ]:
np.isclose(a, b)

In [ ]:
np.isclose(a, b).all()

In [ ]:
np.allclose(a, b)

In [ ]:
np.isclose(a, b).any()

In [ ]:
any([False, False])

**Не используйте** встроенные агрегирующие операции python.

[Первая причина](https://youtu.be/hj_ylt0gq0Y?t=24) – это скорость.

In [ ]:
b = np.random.randint(0, 10, size=(1_000, 1_000))
b[36, 42] = 20
b = b.ravel()

In [ ]:
%%timeit

b.max()

In [ ]:
%%timeit

max(b)

[Вторая причина](https://youtu.be/hj_ylt0gq0Y?t=27) – не совсем очевидное поведение.

In [ ]:
sum(a)  # same as a.sum(axis=0)

Над булевыми массивами можно совершать аггрегирующие логические операции.

In [ ]:
a = np.asarray([[True, True,  False, False, True ],
                [True, False, False, True,  False]])
a

In [ ]:
a.all(), np.all(a)

In [ ]:
a.any(), np.any(a)

In [ ]:
a.all(axis=0)

In [ ]:
a.any(axis=0)

## Индексация

Подробное описание: https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html

**Замечание:** индексация может быть использована не только для получения значений, но и для их присвоения.

### Индексация в одномерных массивах

`numpy.ndarray` поддерживает все те же способы индексации, что и обычный список. 

#### Классическая индексация

In [ ]:
a = np.arange(15)
a

In [ ]:
a[0], a[5], a[len(a) - 1]

#### Отрицательные индексы

In [ ]:
a[len(a) - 1], a[-1]

In [ ]:
a[len(a) - 5], a[-5]

#### Срезы (slices)

**Общее правило:** `массив[первый индекс:последний индекс:шаг]`.

Значения по-умолчанию:
- первый индекс = 0; 
- последний индекс = len(массив);
- шаг = 1;
    
`последний индекс` не включается.

In [ ]:
a = np.arange(15)
a

Взять первые 5 элементов.

In [ ]:
variants = [a[0:5:1], a[0:5], a[:5], ]

print(*map(repr, variants), sep='\n')

In [ ]:
variants = [a[slice(0, 5, 1)], a[slice(0, 5)], a[slice(5)], ]

print(*map(repr, variants), sep='\n')

Взять все элементы, стоящих на четных позициях.

In [ ]:
variants = [a[0:len(a):2], a[0::2], a[::2], ]

print(*map(repr, variants), sep='\n')

Взять все элементы, стоящие на нечетных позициях.

In [ ]:
variants = [a[1:len(a):2], a[1::2], ]

print(*map(repr, variants), sep='\n')

Взять все элементы с 3 по 12 (не включительно) с шагом 3.

In [ ]:
variants = [a[3:12:3], a[3:-3:3], ]

print(*map(repr, variants), sep='\n')

Взять все элементы с 3 по 12 (включительно) с шагом 3 в обратном порядке.

In [ ]:
variants = [a[3:13:3][::-1], a[12:2:-3], a[-3:2:-3], ]

print(*map(repr, variants), sep='\n')

#### Булева индексация (маски)

Маска для массива `a` – булев массив, размер которого совпадает с размером `a`.

In [ ]:
np.random.seed(96)

a = np.random.randint(-5, 7, 34)
a

Найти все отрицательные элементы.

In [ ]:
a < 0

In [ ]:
a[a < 0]

Найти все элементы, кратные 3-м.

In [ ]:
np.random.seed(22)

a = np.random.randint(0, 10, 20)
a

In [ ]:
variants = [
    a[a % 3 == 0],
    a[np.logical_not(a % 3)],
    a[~((a % 3).astype(bool))],
]

print(*map(repr, variants), sep='\n')

Найти все элементы, кратные или 3, или 5.

In [ ]:
mask_3 = a % 3 == 0
mask_5 = a % 5 == 0

variants = [
    a[np.logical_or(mask_3, mask_5)],
    a[mask_3 | mask_5],
    a[(a % 3 == 0) | (a % 5 == 0)],
]

print(*map(repr, variants), sep='\n')

#### Сложная индексация (Fancy indexing)

В массиве `numpy.ndarray` можно выбрать значения, стоящие в нескольких конкретных позициях.

In [ ]:
a = np.arange(10)
a

In [ ]:
a[[2, 7, 5, -1]]

In [ ]:
a[0], a[[0]]

### Индексация в многомерных массивах

`numpy.ndarray` поддерживает все те же способы индексации, описанные выше, относительно строк и столбцов.

In [ ]:
a = np.arange(30).reshape(5, -1)
a

Доступ к конкретным элементам.

In [ ]:
# плохой способ, мы работаем с numpy-массивами, а не со списками

a[0][0], a[0][2], a[1][1], a[-1][-2]

In [ ]:
# хороший способ

a[0, 0], a[0, 2], a[1, 1], a[-1, -2]

Получить строку с индексом 2.

In [ ]:
variants = [a[2], a[2,:], ]

print(*map(repr, variants), sep='\n')

Получить столбец с индексом 3.

In [ ]:
a[:, 3]

Получить все элементы, стоящие в четных столбцах.

In [ ]:
a[:, ::2]

Получить все элементы, стоящие в 0-м столбце и нечетных строках.

In [ ]:
a[1::2, 0]

In [ ]:
a[1::2, [0]]

Получите строки, в которых есть хотя бы один 0.

In [ ]:
np.random.seed(2238)

a = np.random.randint(-5, 5, size=(5, 5))
a

In [ ]:
(a == 0).any(axis=1)

In [ ]:
a[(a == 0).any(axis=1),:]

<font color='red'> **Задача.** Дана матрица $B$ размера $(5 \times 5)$. Получите столбцы, в которых число положительных элементов больше числа отрицательных. Результат сохраните в переменную `B_masked`. Используйте индексацию с использованием булевых массивов.</font>

In [ ]:
np.random.seed(2238)

B = np.random.randint(-5, 5, size=(5, 5))
B

In [ ]:
B_masked = ...

In [ ]:
assert (B_masked == B[:, [0, 1, -1]]).all()

Для получения индексов, где выполнено условие (значение истино) используеттся функция `np.where`. Результат этой функции можно использовать для индексации.

Индексация по результату `np.where` является сложной индексацией (об этом ниже).

In [ ]:
a < 4

In [ ]:
np.where(a < 4)

In [ ]:
a[np.where(a < 4)]

In [ ]:
a[a < 4]

#### Сложная индексация (Fancy indexing)

В массиве `numpy.ndarray` можно выбрать значения, стоящие в нескольких (более 1) конкретных строках ИЛИ столбцах.

In [ ]:
a[[1, 3, 4], :]

По каждой размерности подаются массивы одинаковых размеров, элементы которых соответствуют индексам каждой размерности. Тогда, на выходе будет массив размера этих массивов, элементы которого будут соответствовать элементам исходного массива, взятых в точках, соответствующих поданным массивам.

In [ ]:
a

In [ ]:
a[[0, 1, 1], [2, 1, 2]]

Если требуется взять значения, стоящие в определенных строках и столбцах, то для этого можно использовать функцию `np.ix_`.

In [ ]:
a = np.arange(30).reshape(5, -1)
a

In [ ]:
a[np.ix_([2, 4, 3], [0, -1])]

In [ ]:
a[[2, 4, 3],:][:,[0, -1]]

In [ ]:
a[np.ix_([2, 4, 3], [0, -1])] = 10000
a

In [ ]:
a[[2, 4, 3],:][:,[0, -1]] = 20_000
a

#### Сокращенная индексация

In [ ]:
a = np.arange(24).reshape(2, 3, 4)
a

In [ ]:
a.shape

`...` – константа в Python, которая называется [`Ellipsis`](https://docs.python.org/3/library/constants.html#Ellipsis). 

In [ ]:
a[..., 0]       # same as a[:, :, 0]

In [ ]:
a[0, ...]       # same as a[0, :, :]

In [ ]:
a[0, ..., 0]   # same as a[0, :, 0]

### Другие способы индексации

https://docs.scipy.org/doc/numpy/reference/routines.indexing.html

Посмотрите дома: `np.take_along_axis`, `np.take`.

### Тонкости при индексации (view и копии)

При некоторых видах индексации, получается view (новый объект, который ссылается на те же данные), в других – копия. 

Поподробнее об этом можно прочитать тут:

https://www.jessicayung.com/numpy-views-vs-copies-avoiding-costly-mistakes/

### Конкатенация массивов

In [ ]:
np.random.seed(1398)

c = np.random.permutation(np.arange(0, 12))
i = c.shape[0] // 2

a, b = c[:i], c[i:]
a, b = a.reshape(3, 2), b.reshape(3, 2)

Конкатенация массивов по вертикали.

In [ ]:
print_as_columns(a, b)
np.vstack((a, b))   # same as np.concatenate((a, b), axis=0)

Конкатенация массивов по горизонтали.

In [ ]:
print_as_columns(a, b)
np.hstack((a, b))   # same as np.concatenate((a, b), axis=1)

### Сортировка

In [ ]:
np.random.seed(5555)

a = np.random.randint(0, 10, size=(3, 7))
a[1, 3] = 15

a

In [ ]:
b = a.copy()
b.sort(axis=1)  # inplace
b

In [ ]:
np.sort(a, axis=1)   # new object

И конечно же есть операция `argsort`.

In [ ]:
a.argsort(axis=1)  # same as np.argsort(a, axis=1)

### Получение уникальных элементов

In [ ]:
np.unique(a)

In [ ]:
np.unique(a, return_counts=True)

In [ ]:
np.bincount(a)

In [ ]:
np.bincount(a.ravel())

### Линейная алгебра

https://numpy.org/doc/stable/reference/routines.linalg.html

## Библиотека SciPy

SсiPy — библиотека для сложных научных вычислений в Python, интегрированная с numpy.
В частности, SciPy включает в себя:

* операции линейной алгебры (scipy.linalg);
* реализации методов оптимизации (scipy.optimize);
* статистические критерии и сложные распределения (scipy.stats);
* функции для численного интегрирования (scipy.integrate).

**Важно:** некоторые из операций есть как в numpy, так и в scipy.

**Важно:** scipy — более тяжеловесен чем numpy, т.к. использует внутри себя процедуры языка Fortran.

Более подробный обзор библиотеки можно получить здесь:

http://scipy-lectures.org/intro/scipy.html

### Поиск расстояний между векторами

In [ ]:
from scipy.spatial.distance import cdist, pdist

In [ ]:
np.random.seed(63)

a = np.random.choice(10, size=10, replace=False).reshape(-1, 2)
b = np.random.choice(10, size=10, replace=False).reshape(-1, 2)

In [ ]:
print_as_columns(a, b)

In [ ]:
cdist(a, b)

In [ ]:
cdist(a, b, metric='cosine')

### SVD разложение

In [ ]:
from scipy.linalg import svd

In [ ]:
A = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])

In [ ]:
U, s, Vh = svd(A, full_matrices=True)

In [ ]:
np.allclose(A, U @ np.diag(s) @ Vh)

### Интерполяция

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
np.random.seed(56)

x_data = np.linspace(-5, 5, num=50)
y_data = 2.9 * np.sin(1.5 * x_data) + np.random.normal(size=50)

In [ ]:
plt.plot(x_data, y_data, 'o')

In [ ]:
def fitting_function(x, a, b):
    return a * np.sin(b * x)

In [ ]:
params, params_covariance = curve_fit(fitting_function, x_data, y_data)

In [ ]:
params

In [ ]:
curve_y_data = [fitting_function(x, *params) for x in x_data]

In [ ]:
plt.plot(x_data, y_data, 'o')
plt.plot(x_data, curve_y_data)

### Разреженные матрицы

https://docs.scipy.org/doc/scipy/reference/sparse.html

https://matteding.github.io/2019/04/25/sparse-matrices/

## Что почитать еще?

<table>
    <tr>
        <td><img src="files/book01.jpeg" width="300px"/></td>
        <td><img src="files/book02.jpg" width="300px"/></td>
    </tr>
</table>

1. Jake VanderPlas. Python Data Science Handbook (Вандер Плас Дж. Python для сложных задач.)
[англ.](https://jakevdp.github.io/PythonDataScienceHandbook/)
2. Wes McKinney. Python for Data Analysis (Маккини Уэс. Python и анализ данных)

И последний совет на сегодня...

<center><h3>Не забываем пользоваться поиском, читать документацию и stackoverflow</h3></center>